# Merging non-covid annotations with raw notes

This notebook reads the extracted non-covid annotations (i.e. the output of `annotation_extraction_noncovid.ipynb`). It then matches these notes up with the raw patient records from 2017, using patient and note ID numbers. This **provides the date-time stamp for each note, which is essential for timeseries analysis**. 

Be warned, this notebook shows all of the "working out" steps. There were multiple versions of the original patient records with different structures and no column names. A lot of this notebook is just me trying to figure out which columns are which and match them up with the annotated sentences. A sneaky shift-index-by-one issue made this tricky, but it worked in the end.

In [ ]:
!pwd

import pickle
from pathlib import Path

import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import sklearn
import statsmodels
import torch
from matplotlib import pyplot as plt
from tqdm import tqdm as tqdm

# Make graphics nice
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Set sensible defaults
sns.set()
sns.set_style("ticks")
sns.set_context('paper')

In [2]:
# Read in the annotated non-covid notes we exported in another notebook
df_annot = pd.read_csv('~/gianluca_data/traindata_batch1_v2.tsv', sep='\t')
print(df_annot.shape)

(216959, 7)


In [3]:
# This parses the source filename for the pickled BERTje container, yielding the `src_id`
# field that we need to merge with the raw notes. 

def parse_source(row):
    srcid = str(row.src_file.split("---")[1])
    if np.isnan(row.sentence_id):
        return int(srcid)
    sen = str(int(row.sentence_id))
    srcid = srcid[:-len(sen)]
    return int(srcid)
    
# Parse source file row index (with offset of 1)
df_annot['src_id'] = df_annot.apply(parse_source, axis=1) - 1

In [4]:
df_annot.drop('src_file', axis=1, inplace=True)

In [6]:
col_names = ['id1', 'id2', 'type', 'date', 'note', 'other1', 'other2']

# Read in one set of candidate notes
df_2020vumcnotes = pd.read_csv(
    "//data/Levering COVID project 2020/Notities VUmc.csv", 
    names=col_names,
    sep=';'
)
df_2020vumcnotes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337391 entries, 0 to 337390
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id1     337391 non-null  int64 
 1   id2     337391 non-null  int64 
 2   type    295904 non-null  object
 3   date    337391 non-null  object
 4   note    337198 non-null  object
 5   other1  4491 non-null    object
 6   other2  330 non-null     object
dtypes: int64(2), object(5)
memory usage: 18.0+ MB


In [7]:
# Read in another set of candidate notes (cleaned)
df_notes_cleaned = pd.read_csv(
    "//data/notes/vumc/all_data/notities_2017_deel2_cleaned.csv")
df_notes_cleaned.rename({'Unnamed: 0': 'note_index'}, axis=1, inplace=True)
df_notes_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1249009 entries, 0 to 1249008
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   note_index  1249009 non-null  int64 
 1   notitieID   1249009 non-null  int64 
 2   notitie     1249009 non-null  object
dtypes: int64(2), object(1)
memory usage: 28.6+ MB


In [9]:
# Read in the original raw notes (not cleanded)
df_notes_orig = pd.read_csv(
    "//data/notes/vumc/all_data/notities_2017_deel2.csv",
    names=col_names,
    sep=';',
)
df_notes_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1249545 entries, 0 to 1249544
Data columns (total 7 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   id1     1249545 non-null  object
 1   id2     1249545 non-null  int64 
 2   type    1028367 non-null  object
 3   date    1249545 non-null  object
 4   note    1249010 non-null  object
 5   other1  14928 non-null    object
 6   other2  899 non-null      object
dtypes: int64(1), object(6)
memory usage: 66.7+ MB


In [12]:
df_notes_orig.shape, df_notes_cleaned.shape

((1249545, 7), (1249009, 3))

In [13]:
# Merge the original and cleaned notes with an inner join
df_notes = df_notes_orig.merge(df_notes_cleaned, how='inner', left_on='id2', right_on='notitieID')
df_notes.reset_index(inplace=True, drop=True)
df_notes.shape

(1249009, 10)

In [ ]:
# Rename columns, drop some redundant ones, and then reorder the columns for better readability.
df_notes.rename({'id1': 'patient_id', 'id2': 'note_id'}, axis=1, inplace=True)
df_notes.drop(['other1', 'other2', 'notitieID', 'notitie'], axis=1, inplace=True)
df_notes = df_notes[['note_id', 'patient_id', 'date', 'type', 'note_index', 'note']]
df_notes['date'] = pd.to_datetime(df_notes['date'])

# Redacted for patient privacy
df_notes.sample(5)

In [15]:
df_notes.date.describe(datetime_is_numeric=True)

count                          1249009
mean     2017-08-08 19:42:52.004204544
min                2017-01-01 00:00:00
25%                2017-06-06 00:00:00
50%                2017-08-23 00:00:00
75%                2017-10-27 00:00:00
max                2017-12-31 00:00:00
Name: date, dtype: object

In [16]:
df_2020vumcnotes['date'] = pd.to_datetime(df_2020vumcnotes['date'])
df_2020vumcnotes['date'].describe(datetime_is_numeric=True)

count                           337391
mean     2018-07-08 06:55:47.321060352
min                2018-01-01 00:00:00
25%                2018-04-09 00:00:00
50%                2018-07-10 00:00:00
75%                2018-10-09 00:00:00
max                2018-12-31 00:00:00
Name: date, dtype: object

In [ ]:
df_2020vumcnotes = df_2020vumcnotes.reset_index(drop=False)
df_2020vumcnotes.rename({"index": 'note_index'}, axis=1, inplace=True)
df_2020vumcnotes.drop(['other1', 'other2'], axis=1, inplace=True)
df_2020vumcnotes.head(3)

In [20]:
# Hunting to find which of the (unknown) columns match up
print(np.intersect1d(df_2020vumcnotes.id2.unique(), df_notes.note_id.unique()).size)
print(np.intersect1d(df_2020vumcnotes.id1.unique(), df_notes.note_id.unique()).size)
df_2020vumcnotes.id2.unique().size, df_notes.note_id.unique().size

0
0


(337391, 1249009)

In [21]:
df_notes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1249009 entries, 0 to 1249008
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   note_id     1249009 non-null  int64         
 1   patient_id  1249009 non-null  object        
 2   date        1249009 non-null  datetime64[ns]
 3   type        1027884 non-null  object        
 4   note_index  1249009 non-null  int64         
 5   note        1249009 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 57.2+ MB


In [24]:
df_notes.note_index.min(), df_notes.note_index.max()

(0, 1249008)

In [26]:
df_annot.src_id.min(), df_annot.src_id.max()

(15, 37564)

In [27]:
df_notes.note_index.min(), df_notes.note_index.max()

(0, 1249008)

In [28]:
np.intersect1d(df_annot.src_id.unique(), df_notes.note_index.unique()).size

2785

In [29]:
df_annot.src_id.nunique(), df_notes.note_index.nunique()

(2785, 1249009)

In [30]:
np.intersect1d(df_annot.src_id.unique(), df_notes.note_index.unique()).size / df_annot.src_id.nunique()

1.0

In [31]:
df_annot.src_id.value_counts().head()

14057    669
20516    660
22104    642
10605    628
2531     582
Name: src_id, dtype: int64

## Where things actually start working

In [32]:
# Merge and save the final dataframe
df_merged = df_annot.merge(df_notes, how='outer', left_on='src_id', right_on='note_index')
df_merged.to_csv('~/gianluca_data/traindata_batch1_v2_matched.tsv', sep='\t', index=False)
df_merged.shape

(1463183, 13)

In [33]:
df_annot.shape, df_notes.shape

((216959, 7), (1249009, 6))

In [34]:
df_merged.note_id.value_counts().head()

209204887    669
222899243    660
221352103    642
207579644    628
170720974    582
Name: note_id, dtype: int64

In [ ]:
# A sanity check on a particular note. Output redacted for patient privacy.
print(df_merged[df_merged.src_id == 2531].note.values[0][:500])
print(df_merged[df_merged.src_id == 2531].sentence.values[:20])

In [36]:
# Count what fraction of the notes contain the annotated sentence
def sent_in_note(s, n):
    return 1 if str(s) in str(n) else 0

df_merged['sent_in_note'] = df_merged.apply(lambda x: sent_in_note(x.sentence, x.note), axis=1)
df_merged['sent_in_note'].sum() / df_merged.shape[0]

0.06707500018794642